In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from numpy import linalg as la
import sys
import matplotlib.pyplot as plt
import seaborn as sas
import os
import csv
import re
import sklearn.preprocessing as pre
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [2]:
# dict Position, file
label_Exp1 = {
    '1': 'Supine',
    '2': 'Right',
    '3': 'Left',
    '4': 'Right',
    '5': 'Right',
    '6': 'Left',
    '7': 'Left',
    '8': 'Supine',
    '9': 'Supine',
    '10': 'Supine',
    '11': 'Supine',
    '12': 'Supine',
    '13': 'Right',
    '14': 'Left',
    '15': 'Supine',
    '16': 'Supine',
    '17': 'Supine'
}

label_Exp2 = {
    'B1': 'Supine',
    'B2': 'Supine',
    'B3': 'Supine',
    'B4': 'Supine',
    'B5': 'Supine',
    'B6': 'Supine',
    'B7': 'Supine',
    'B8': 'Supine',
    'B9': 'Supine',
    'B10': 'Supine',
    'C1': 'Right',
    'C2': 'Right',
    'C3': 'Right',
    'D1': 'Left',
    'D2': 'Left',
    'D3': 'Left',
    'E1': 'Right',
    'E2': 'Right',
    'E3': 'Left',
    'E4': 'Left',
    'E5': 'Right',
    'E6': 'Left',
    'F1': 'Supine',
    'F2': 'Supine',
    'F3': 'Supine',
    'F4': 'Supine',
    'F5': 'Supine',
    'F6': 'Supine',
    'F7': 'Supine'
}


In [3]:
class Dataset:
    def __init__(self, high_x, high_y, high_average, center_x, center_y, center_average, low_x, low_y, low_average, subject, position):
        self.high_x = high_x
        self.high_y = high_y
        self.high_average = high_average
        self.center_x = center_x
        self.center_y = center_y
        self.center_average = center_average
        self.low_x = low_x
        self.low_y = low_y
        self.low_average = low_average
        self.subject = subject
        self.position = position
        

In [4]:
def PCA(file):
    df = pd.read_csv(file, sep="\t", header=None)
    # measure same position 82 times DC
    df.drop(columns=[2048], inplace = True) #tolgo colonna
    df.drop(0, inplace = True) #tolgo riga
    df.drop(1, inplace = True) #tolgo riga
    df.drop(2, inplace = True) #tolgo riga
    
    array = df.to_numpy()
    avg = np.mean(array, axis = 1, keepdims=True)

    array = array - avg

    CX = np.cov(array)
    w,v = la.eig(CX) 

    CY =  np.diag(np.round(np.real(w),2))

    P = v[:,0:5]
    P = abs(P)

    array_final = P.T.dot(array)
        
    return array_final
    
def padding(old_map):
    start = np.zeros((64, 3))
    final = np.zeros((64, 2))
    new_map = np.hstack((start, old_map, final))
    
    return new_map

def flip(pressure_map, experiment):
    
    if experiment == 1:
        pressure_map = np.flip(pressure_map, 1)
    else:
        pressure_map = np.flip(pressure_map, 0)
    
    return pressure_map
    
def trueLabel(num_file, experiment):

    if experiment == 1:
        position = label_Exp1.get(num_file)
    else: 
        position = label_Exp2.get(num_file)
        
    return position

def clean_string(string, experiment):
    if experiment == 1:
        file = string[24:27]
        file = re.sub('[./_t]', '', file)
    
    else:
        if 'Air' in string:
            file = string[44:48]
            file = re.sub('[./_t]', '', file)
        else:
            file = string[50:54]
            file = re.sub('[./_t]', '', file)

    return file

def label_position(position):
    if position == 'Supine':
        return 0
    
    if position == 'Right':
        return 1
    
    if position == 'Left':
        return 2

def rescale(pressure_map, max_value, min_value):

    minimum = pressure_map.min()  
    maximum = pressure_map.max()
    OldRange = maximum - minimum  
    NewRange = max_value - min_value

    for i in range(0,len(pressure_map)):
        pressure_map[i] = (((pressure_map[i] - minimum) * NewRange) / OldRange) 
            
    return pressure_map

def kMeans(section):

    X = []
    
    for i in range(0, section.shape[0]):
        for j in range(0, section.shape[1]):
            t = []
            t.append(i)
            t.append(j)
            t.append(section[i][j])
            X.append(t)

    X = np.array(X)

    kmeans = KMeans(n_clusters=2).fit(X)
    
    # compute x, y centroid and avg intensity in cluster
    cluster_1 = []
    cluster_2 = []

    for i in range(0,len(X)):
        if kmeans.labels_[i] == 0:
            cluster_1.append(X[i])
        else:
            cluster_2.append(X[i])

    mean_1 = np.mean(cluster_1, axis = 0) 
    mean_2 = np.mean(cluster_2, axis = 0) 

    if mean_1[2] > mean_2[2]:
        mean_cluster = mean_1[2]
        y_pos = kmeans.cluster_centers_[0][0]
        x_pos = kmeans.cluster_centers_[0][1]
    else:
        mean_cluster = mean_2[2]
        y_pos = kmeans.cluster_centers_[1][0]
        x_pos = kmeans.cluster_centers_[1][1]

    
    return x_pos, y_pos, mean_cluster


In [5]:
# collects data from experiment 1
rootdir = Path("dataset/experiment-i/")
exp = 1

pressure_map = []
dataset = []

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        #print(os.path.join(subdir, file))
        file_name = os.path.join(subdir, file)
        
        pressure = PCA(file_name)
           
        for i in range(0, len(pressure)):
            
            subject = file_name[22:24]
            if subject[1:2] == '/':
                subject = subject[0]

            file = clean_string(file_name, exp)
            position = trueLabel(file, exp)
            position = label_position(position)
        
            pressure_map = flip(pressure[i].reshape((64, 32)), exp)
            pressure_map = rescale(pressure_map, 500, 0)
            
            section1 = pressure_map[:14]
            section2 = pressure_map[14:37]
            section3 = pressure_map[37:]
            pos_h_x, pos_h_y, mean_h = kMeans(section1)
            pos_c_x, pos_c_y, mean_c = kMeans(section2)
            pos_l_x, pos_l_y, mean_l = kMeans(section3)
            
            data = Dataset(pos_h_x, pos_h_y, mean_h, pos_c_x, pos_c_y + 14, mean_c, pos_l_x, pos_l_y + 37, mean_l, int(subject)-1, int(position))
            dataset.append(data)
        

In [6]:
# collects data from experiment 2
rootdir = Path("dataset/experiment-ii/")
exp = 2

pressure_map = []

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        #print(os.path.join(subdir, file))
        file_name = os.path.join(subdir, file)
        
        #TODO: remove this if
        if '.DS_Store' in file_name:
            continue
        
        df = pd.read_csv(file_name, sep=" ", header=None)
        pressure_map = df.to_numpy()
        
        pressure_map = flip(pressure_map, exp)
        pressure_map = padding(pressure_map)
        pressure_map = rescale(pressure_map, 500, 0)
        
        section1 = pressure_map[:14]
        section2 = pressure_map[14:37]
        section3 = pressure_map[37:]
        pos_h_x, pos_h_y, mean_h = kMeans(section1)
        pos_c_x, pos_c_y, mean_c= kMeans(section2)
        pos_l_x, pos_l_y, mean_l = kMeans(section3)

        subject = file_name[23:25]
        if subject[1:2] == '/':
                subject = subject[0]
        
        file = clean_string(file_name, exp)
        position = trueLabel(file, exp)
        position = label_position(position)
        
        data = Dataset(pos_h_x, pos_h_y, mean_h, pos_c_x, pos_c_y + 14, mean_c, pos_l_x, pos_l_y + 37, mean_l, int(subject)-1, int(position))
        dataset.append(data)


In [7]:
# write in csv
with open('dataset_2.csv', mode='w') as csv_file:
    fieldnames = ['Head X', 'Head Y', 'Head Pressure', 'Hip Bone X', 'Hip Bone Y', 'Hip Bone Pressure', 'Legs X', 'Legs Y', 'Legs Pressure', 'Subject', 'Position']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    writer.writeheader()
    for data in dataset:
        writer.writerow({'Head X': data.high_x, 'Head Y': data.high_y, 'Head Pressure': data.high_average, 'Hip Bone X': data.center_x, 'Hip Bone Y': data.center_y, 'Hip Bone Pressure': data.center_average, 'Legs X': data.low_x, 'Legs Y': data.low_y, 'Legs Pressure': data.low_average, 'Subject': data.subject, 'Position': data.position})

